Trước tiên chúng ta sẽ nạp bộ dữ liệu để train từ folder train

In [10]:

# Deep Learning CNN model to recognize face
'''This script uses a database of images and creates CNN model on top of it to test
   if the given image is recognized correctly or not'''
 
'''####### IMAGE PRE-PROCESSING for TRAINING and TESTING data #######'''
 
# Specifying the folder where images are present
TrainingImagePath='./train'
 
from keras.preprocessing.image import ImageDataGenerator
# Understand more about ImageDataGenerator at below link
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
 
# Defining pre-processing transformations on raw images of training data
# These hyper parameters helps to generate slightly twisted versions
# of the original image, which leads to a better model, since it learns
# on the good and bad mix of images
train_datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)
 
# Defining pre-processing transformations on raw images of testing data
# No transformations are done on the testing images
test_datagen = ImageDataGenerator()
 
# Generating the Training Data
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        color_mode = "grayscale",
        batch_size=1,
        class_mode='categorical')
 
 
# Generating the Testing Data
test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        color_mode = "grayscale",
        batch_size=1,
        class_mode='categorical')
 
# Printing class labels for each face
test_set.class_indices

Found 457 images belonging to 22 classes.
Found 457 images belonging to 22 classes.


{'Hưng': 0,
 'Hồng': 1,
 'Nhân': 2,
 'Thịnh': 3,
 'Tài': 4,
 'face1': 5,
 'face10': 6,
 'face11': 7,
 'face12': 8,
 'face13': 9,
 'face14': 10,
 'face15': 11,
 'face16': 12,
 'face2': 13,
 'face3': 14,
 'face4': 15,
 'face5': 16,
 'face6': 17,
 'face7': 18,
 'face8': 19,
 'face9': 20,
 'unknown': 21}

Đoạn dưới là đánh dấu từng nhãn theo dictionary rồi lưu vào file cho trường hợp sau này muốn test model không cần chạy lại đoạn ở trên cũng như ở dưới đây

In [11]:
# class_indices have the numeric tag for each face
TrainClasses=training_set.class_indices
 
# Storing the face and the numeric tag for future reference
ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    print(faceValue,' : ',faceName)
    ResultMap[faceValue]=faceName
 
# Saving the face map for future reference
import pickle
with open("ResultsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)
 
# The model will give answer as a numeric tag
# This mapping will help to get the corresponding face name for it
print("Mapping of Face and its ID",ResultMap)
 
# The number of neurons for the output layer is equal to the number of faces
OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

0  :  Hưng
1  :  Hồng
2  :  Nhân
3  :  Thịnh
4  :  Tài
5  :  face1
6  :  face10
7  :  face11
8  :  face12
9  :  face13
10  :  face14
11  :  face15
12  :  face16
13  :  face2
14  :  face3
15  :  face4
16  :  face5
17  :  face6
18  :  face7
19  :  face8
20  :  face9
21  :  unknown
Mapping of Face and its ID {0: 'Hưng', 1: 'Hồng', 2: 'Nhân', 3: 'Thịnh', 4: 'Tài', 5: 'face1', 6: 'face10', 7: 'face11', 8: 'face12', 9: 'face13', 10: 'face14', 11: 'face15', 12: 'face16', 13: 'face2', 14: 'face3', 15: 'face4', 16: 'face5', 17: 'face6', 18: 'face7', 19: 'face8', 20: 'face9', 21: 'unknown'}

 The Number of output neurons:  22


Tạo model CNN gồm 4 lớp hidden layers , 22 đầu ra , activation là Leaky Relu , áp dụng Batch Normalization và 
dropout để giảm over fitting

In [15]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense,LeakyReLU,BatchNormalization,Dropout
 
'''Initializing the Convolutional Neural Network'''
classifier= Sequential()
 
''' STEP--1 Convolution
# Adding the first layer of CNN
# we are using the format (64,64,3) because we are using TensorFlow backend
# It means 3 matrix of size (64X64) pixels representing Red, Green and Blue components of pixels
'''
classifier.add(Convolution2D(64, kernel_size=(3, 3), strides=(1, 1), input_shape=(64,64,1), activation= LeakyReLU(alpha=0.1)))
classifier.add(BatchNormalization())
'''# STEP--2 MAX Pooling'''
classifier.add(MaxPool2D(pool_size=(2,2)))
classifier.add(Dropout(0.25))
 
'''############## ADDITIONAL LAYER of CONVOLUTION for better accuracy #################'''
classifier.add(Convolution2D(128, kernel_size=(3, 3), strides=(1, 1),  activation= LeakyReLU(alpha=0.2)))
classifier.add(BatchNormalization())
classifier.add(MaxPool2D(pool_size=(2,2)))
classifier.add(Dropout(0.25))
 
classifier.add(Convolution2D(256, kernel_size=(3, 3), strides=(1, 1),  activation= LeakyReLU(alpha=0.2)))
classifier.add(BatchNormalization())
classifier.add(MaxPool2D(pool_size=(2,2)))
classifier.add(Dropout(0.25))

classifier.add(Convolution2D(512, kernel_size=(3, 3), strides=(1, 1),  activation= LeakyReLU(alpha=0.3)))
classifier.add(BatchNormalization())
classifier.add(MaxPool2D(pool_size=(2,2)))
classifier.add(Dropout(0.25))

'''# STEP--3 FLattening'''
classifier.add(Flatten())
 
'''# STEP--4 Fully Connected Neural Network'''
classifier.add(Dense(64, activation='relu'))
 
classifier.add(Dense(OutputNeurons, activation='softmax'))
 
'''# Compiling the CNN'''
#classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])
 

Đoạn code ở dưới chỉ chạy sau khi đã train xong model trước đó và muốn thử kết quả , không cần chạy lại những đoạn code ở trên

In [2]:
from keras.models import load_model
import pickle
classifier = load_model('./train-weight/21out_v4.h5') # sửa tên file .h5 cũng như đường dẫn nếu thay bằng bộ W khác
infile = open('./ResultsMap.pkl','rb')
ResultMap = pickle.load(infile)
infile.close()

In [16]:

###########################################################
import time
# Measuring the time taken by the model to train
StartTime=time.time()
 
# Starting the model training
classifier.fit(
                   training_set,
                    steps_per_epoch=training_set.n//training_set.batch_size,
                    epochs=80,
                    validation_data=test_set,
                    validation_steps=test_set.n//test_set.batch_size)
 
EndTime=time.time()
print("###### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ######')

Epoch 1/80
457/457 [==============================] - 57s 121ms/step - loss: 2.6914 - accuracy: 0.2035 - val_loss: 2.4733 - val_accuracy: 0.2451
Epoch 2/80
457/457 [==============================] - 57s 125ms/step - loss: 2.3339 - accuracy: 0.2451 - val_loss: 2.4079 - val_accuracy: 0.2319
Epoch 3/80
457/457 [==============================] - 58s 127ms/step - loss: 2.2205 - accuracy: 0.3129 - val_loss: 2.1101 - val_accuracy: 0.3151
Epoch 4/80
457/457 [==============================] - 63s 139ms/step - loss: 2.1875 - accuracy: 0.3129 - val_loss: 2.1204 - val_accuracy: 0.3479
Epoch 5/80
457/457 [==============================] - 57s 124ms/step - loss: 2.0272 - accuracy: 0.3414 - val_loss: 1.9995 - val_accuracy: 0.3632
Epoch 6/80
457/457 [==============================] - 57s 125ms/step - loss: 2.0174 - accuracy: 0.3523 - val_loss: 1.9416 - val_accuracy: 0.3939
Epoch 7/80
457/457 [==============================] - 60s 131ms/step - loss: 1.9673 - accuracy: 0.3523 - val_loss: 2.0356 - val_ac

Dự đoán model

In [5]:

'''########### Making single predictions ###########'''
import numpy as np
import keras.utils as image
 
#ImagePath='./Face Images/Final Testing Images/anh-hung/hung5.jpg'
ImagePath = './test/hong1.jpg'
test_image=image.load_img(ImagePath,color_mode = "grayscale",target_size=(64, 64))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result = classifier.predict(test_image,verbose=0)
#print(training_set.class_indices)
print(np.argmax(result))
 
print('####'*10)
print('Prediction is: ',ResultMap[np.argmax(result)])

1
########################################
Prediction is:  Hồng


Lưu model (Nên đổi tên file)

In [29]:
classifier.save('./train-weight/21out_v4.h5')

In [ ]:
#!pip install -q pyyaml h5py